# Chapter 8. Poisson Processes

[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap08.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
include("./pmfAndCdf.jl")
#import .ProbabilityMassFunction as Pmf # if defined with module (and export) in pmfAndCdf.jl
#using .ProbabilityMassFunction # if defined with module and export in pmfAndCdf.jl

include("./simplestat.jl")
#import .SimpleStatistics as Ss # if defined with module (and export) in simplestat.jl
#using .ProbabilityMassFunction # if defined with module and export in simplestat.jl

### The World Cup Problem

In the 2018 FIFA World Cup final, France defeated Croatia 4 goals to 2. Based on this outcome:
- How confident should we be that France is the better team?
- If the same teams played again, what is the chance France would win again?

To answer these questions, we have to make some modeling decisions.
- First, I’ll assume that for any team against another team there is some unknown goal-scoring rate, measured in goals per game, which I’ll denote with the Python variable `lam` or the Greek letter $\lambda$, pronounced “lambda”.
- Second, I’ll assume that a goal is equally likely during any minute of a game. So, in a 90 minute game, the probability of scoring during any minute is $\lambda / 90$.
- Third, I’ll assume that a team never scores twice during the same minute."

## The Poisson Distribution

If the number of goals scored in a game follows a Poisson distribution with a goal-scoring rate $\lambda$, the probability of scoring $k$ goals is

$\lambda^{k} * exp(-\lambda)/k!$

or

$\frac{\lambda^{k} * exp^{-\lambda}}{k!}$

or

$\frac{\lambda^{k}}{k!}e^{-\lambda}$

for any non-negative value of $k$.

In [ ]:
lam = 1.4
distPois = Dsts.Poisson(lam)

In [ ]:
# k goals
k = 4
Dsts.pdf(distPois, k)

This result implies that if the average goal-scoring rate is 1.4 goals per game, then probability of scoring 4 goals in a game is ~4%.

In [ ]:
# custom type
Num = Union{Int, Float64}

In [ ]:
function getPoisPmf(
        lam::A,
        qs::Vector{B}
        )::Pmf{B} where {A<:Num, B<:Num}
    ps::Vector{Float64} = Dsts.pdf.(Dsts.Poisson(lam), qs)
    pmf::Pmf{B} = Pmf(qs, ps)
    return pmf
end

In [ ]:
lam = 1.4
goals = 0:10 |> collect
pmfGoals = getPoisPmf(lam, goals)

In [ ]:
fig = Cmk.Figure()
ax, bp = Cmk.barplot(fig[1, 1],
    pmfGoals.names, pmfGoals.priors,
    axis=(;title="Distribution of goals scored",
        xlabel="Number of goals",
        ylabel="PMF",
        xticks=0:10
    )
)
Cmk.axislegend(
    ax,
    [bp],
    ["λ = 1.4"],
    "Poisson distribution"
)
fig

[...]

Now let’s turn it around: given a number of goals, what can we say about the goal-scoring rate?

To answer that, we need to think about the prior distribution of `lam`, which represents the range of possible values and their probabilities before we see the score.

## The Gamma Distribution

Using [data](https://www.statista.com/statistics/269031/goals-scored-per-game-at-the-fifa-world-cup-since-1930/) [...] each team scores about 1.4 goals per game, on average (so $\lambda = 1.4$).

For a good team against a bad one, $\lambda$ will be higher; for a bad team against a good one, it will be lower.

We will model it with [gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution). It is usually used to model the time that passes before $\alpha$ occurences of a randomly occuring event (an event that occurs via Poisson process).

In [ ]:
alpha = 1.4
qs = range(0, 10, 101)
ps = Dsts.pdf.(Dsts.Gamma(alpha), qs);

The parameter, `alpha`, is the mean of the distribution. The `qs` are possible values of `lam` between 0 and 10. The `ps` are probability densities, which we can think of as unnormalized probabilities.

In [ ]:
lambdasPmf = Pmf(qs |> collect, ps)

In [ ]:
fig = Cmk.Figure()
Cmk.lines(fig[1, 1],
    lambdasPmf.names, lambdasPmf.priors,
    color="lightgray", linestyle=:dash, linewidth=3,
    axis=(;title="Prior distribution of λ",
        xlabel="Goal scoring rate (λ)", ylabel="PMF",
        xticks=0:2:10, yticks=0:0.01:0.06)
)
fig

This distribution represents our prior knowledge about goal scoring: 'lam' is usually less than 2, occasionally as high as 6, and seldom higher than that.

We can confirm that mean is about 1.4.

In [ ]:
getMean(lambdasPmf, true),
getQuantile(lambdasPmf, 0.5, true),
getCredibleInterval(lambdasPmf, 0.5, true)

[...] could disagree about [...] the prior, but this is good enough to get started.

## The Update

Given the goal-scoring rate, $\lambda$, what is the probability of scoring `k` goals?

In [ ]:
lam = 1.4
k = 4
Dsts.pdf(Dsts.Poisson(lam), k)

Now suppose we have an array of possible values for $\lambda$ we can compute the likelihood of the data for each hypothetical $\lambda$

In [ ]:
likelihoods = Dsts.pdf.(Dsts.Poisson.(lambdasPmf.names), k)

We can encapsulate it into a function

In [ ]:
# pmfPoisson.names = lambdas for Poisson distribution
# pmfPoisson.priors = probabilities for the lambdas
function updatePoisson!(pmfPoisson::Pmf{Float64}, data::Num)
    k::Num = data
    lams::Vector{Float64} = pmfPoisson.names
    likelihoods::Vector{Float64} = Dsts.pdf.(Dsts.Poisson.(lams), k)
    setLikelihoods!(pmfPoisson, likelihoods)
    updatePosteriors!(pmfPoisson, true)
    return nothing
end

In [ ]:
# in the example, France scored 4 goals
france = Pmf(lambdasPmf.names |> copy, lambdasPmf.priors |> copy)
updatePoisson!(france, 4)

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1],
    france.names, france.priors,
    color="lightgray", linestyle=:dash, linewidth=3,
    axis=(;title="Prior distribution of λ",
        xlabel="Goal scoring rate (λ)", ylabel="PMF",
        xticks=0:2:10, yticks=0:0.01:0.06)
)
l2 = Cmk.lines!(fig[1, 1],
    france.names, france.posteriors,
    color="orange", linestyle=:solid, linewidth=3
)
Cmk.axislegend(
    ax1,
    [l1, l2],
    ["prior (average team)", "France posterior"],
    "Poisson distribution"
)
fig

In [ ]:
# Croatia scored 2 goals, so
croatia = Pmf(lambdasPmf.names |> copy, lambdasPmf.priors |> copy)
updatePoisson!(croatia, 2)

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1],
    croatia.names, croatia.priors,
    color="lightgray", linestyle=:dash, linewidth=3,
    axis=(;title="Prior distribution of λ",
        xlabel="Goal scoring rate (λ)", ylabel="PMF",
        xticks=0:2:10, yticks=0:0.01:0.06)
)
l2 = Cmk.lines!(fig[1, 1],
    croatia.names, croatia.posteriors,
    color="blue", linestyle=:solid, linewidth=3
)
Cmk.axislegend(
    ax1,
    [l1, l2],
    ["prior (average team)", "Croatia posterior"],
    "Poisson distribution"
)
fig

In [ ]:
getMean(france),
getMean(croatia)

## Probability of Superiority

How confident should we be that France is the better team?

In the model, “better” means having a higher goal-scoring rate against the opponent. We can use the posterior distributions to compute the probability that a random value drawn from France’s distribution exceeds a value drawn from Croatia’s.

One way to do that is to enumerate all pairs of values from the two distributions, adding up the total probability that one value exceeds the other.

In [ ]:
# prob pmf1 > pmf2
function getProbGt(pmf1::Pmf{A}, pmf2::Pmf{B},
        usePriors::Bool = false)::Float64 where {A<:Num, B<:Num} 
    total::Float64 = 0
    for (n1, p1) in zip(pmf1.names, 
            usePriors ? pmf1.priors : pmf1.posteriors)
        for (n2, p2) in zip(pmf2.names,
                usePriors ? pmf2.priors : pmf2.posteriors)
            if n1 > n2
                total += p1 * p2
            end
        end
    end
    return total
end

In [ ]:
getProbGt(france, croatia)

Of course, we should remember that this result is based on the assumption that the goal-scoring rate is constant. In reality, if a team is down by one goal, they might play more aggressively toward the end of the game, making them more likely to score, but also more likely to give up an additional goal.

## Predicting the Rematch

If the same teams played again, what is the chance Croatia would win? To answer this question, we’ll generate the “posterior predictive distribution”, which is the number of goals we expect a team to score.

If we knew the goal scoring rate, `lam`, the distribution of goals would be a Poisson distribution with parameter `lam`. Since we don’t know `lam`, the distribution of goals is a mixture of a Poisson distributions with different values of `lam`.

In [ ]:
goals = 0:10 |> collect
pmfSeq = [getPoisPmf(lam, goals)
    for lam in lambdasPmf.names];

In [ ]:
fig = Cmk.Figure()
inds = 11:10:41 |> collect
i = 0
for r in 1:2
    for c in 1:2
        i += 1
        ax, bp = Cmk.barplot(fig[r, c],
            pmfSeq[inds[i]].names,
            pmfSeq[inds[i]].priors,
            axis=(;
                title="Goals distribution",
                xlabel="Number of goals",
                ylabel="PMF",
                xticks=0:2:10
            )
        )
        Cmk.axislegend(ax,
            [bp],
            ["λ = $(lambdasPmf.names[inds[i]])"]
        )
    end
end
fig

In [ ]:
francePmfsMixture = getMixture(
    france, pmfSeq,
    map(x -> string("λ = ", x), lambdasPmf.names),
    0:10 |> collect,
    false
)

In [ ]:
fig = Cmk.Figure()
ax, bp = Cmk.barplot(fig[1, 1],
    francePmfsMixture.outcome, francePmfsMixture.posteriors,
    axis=(;title="Posterior predictive distribution",
        xlabel = "Number of goals",
        ylabel = "PMF",
        xticks=0:10
    )
)
Cmk.axislegend(ax,
    [bp],
    ["France"]
)
fig

This distribution represents two sources of uncertainty: we don’t know the actual value of `lam`, and even if we did, we would not know the number of goals in the next game.

Here’s the predictive distribution for Croatia.

In [ ]:
croatiaPmfsMixture = getMixture(
    croatia, pmfSeq,
    map(x -> string("λ = ", x), lambdasPmf.names),
    0:10 |> collect,
    false
)

In [ ]:
fig = Cmk.Figure()
ax, bp = Cmk.barplot(fig[1, 1],
    croatiaPmfsMixture.outcome, croatiaPmfsMixture.posteriors,
    axis=(;title="Posterior predictive distribution",
        xlabel = "Number of goals",
        ylabel = "PMF",
        xticks=0:10
    )
)
Cmk.axislegend(ax,
    [bp],
    ["Croatia"]
)
fig

We can use these distributions to compute the probability that France wins, loses, or ties the rematch.

In [ ]:
predFrance = Pmf(francePmfsMixture.outcome, francePmfsMixture.posteriors)
predCroatia = Pmf(croatiaPmfsMixture.outcome, croatiaPmfsMixture.posteriors);

In [ ]:
pFranceWin = getProbGt(predFrance, predCroatia, true)

In [ ]:
# prob pmf1 < pmf2
function getProbLt(pmf1::Pmf{A}, pmf2::Pmf{B},
        usePriors::Bool = false)::Float64 where {A<:Num, B<:Num} 
    total::Float64 = 0
    for (n1, p1) in zip(pmf1.names, 
            usePriors ? pmf1.priors : pmf1.posteriors)
        for (n2, p2) in zip(pmf2.names,
                usePriors ? pmf2.priors : pmf2.posteriors)
            if n1 < n2
                total += p1 * p2
            end
        end
    end
    return total
end

In [ ]:
pFranceLose = getProbLt(predFrance, predCroatia, true)

In [ ]:
# prob pmf1 == pmf2
function getProbEq(pmf1::Pmf{A}, pmf2::Pmf{B},
        usePriors::Bool = false)::Float64 where {A<:Num, B<:Num} 
    total::Float64 = 0
    for (n1, p1) in zip(pmf1.names, 
            usePriors ? pmf1.priors : pmf1.posteriors)
        for (n2, p2) in zip(pmf2.names,
                usePriors ? pmf2.priors : pmf2.posteriors)
            if isapprox(n1, n2)
                total += p1 * p2
            end
        end
    end
    return total
end

In [ ]:
pFranceTie = getProbEq(predFrance, predCroatia, true)

In [ ]:
pFranceWin + pFranceLose + pFranceTie # should add up to 1

Assuming that France wins half of the ties, their chance of winning the rematch is about

In [ ]:
pFranceWin + pFranceTie/2

This is a bit lower than their probability of superiority, which is 75%. And that makes sense, because we are less certain about the outcome of a single game than we are about the goal-scoring rates. Even if France is the better team, they might lose the game.

## The Exponential Distribution

> In the 2014 FIFA World Cup, Germany played Brazil in a semifinal match. Germany scored after 11 minutes and again at the 23 minute mark. At that point in the match, how many goals would you expect Germany to score after 90 minutes? What was the probability that they would score 5 more goals (as, in fact, they did)?

In this version, notice that the data is not the number of goals in a fixed period of time, but the time between goals.

We will compute that using [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution)

if the goal-scoring is $\lambda$, the probability of seeing an interval between goals of `t` is proportional to PDF of the exponential distribution:

$\lambda * exp(-\lambda * t)$

`Distributions` got `Exponential` distributuon but it does not take `lam` so work with it is not stragithforward. Therefore our own function

In [ ]:
function getExpoPdf(t::A, lam::B)::Float64 where {A<:Num, B<:Num}
    return lam * exp(-lam * t)
end

To see what the exponential distribution looks like, let’s assume again that `lam` is 1.4

In [ ]:
lam = 1.4
qs = range(0, 4, 101) |> collect
ps = getExpoPdf.(qs, lam)
pmfTime = Pmf(qs, ps);

In [ ]:
fig = Cmk.Figure()
ax, ln = Cmk.lines(fig[1, 1],
    pmfTime.names, pmfTime.priors,
    axis=(;title="Distribution of time between goals",
        xlabel="Time between goals (games)",
        ylabel="PMF",
        xticks=0:0.5:4,
        yticks=0:0.01:0.06
    )
)
Cmk.axislegend(ax,
    [ln],
    ["exponential with λ = 1.4"]
)
fig

It is counterintuitive, but true, that the most likely time to score a goal is immediately. After that, the probability of each successive interval is a little lower.

With a goal-scoring rate of 1.4, it is possible that a team will take more than one game to score a goal, but it is unlikely that they will take more than two games.

## Summary

This chapter introduces three new distributions [...]. Let’s review:
- If a system satisfies the assumptions of a Poisson model, the number of events in a period of time follows a Poisson distribution, which is a discrete distribution with integer quantities from 0 to infinity. In practice, we can usually ignore low-probability quantities above a finite limit.

- Also under the Poisson model, the interval between events follows an exponential distribution, which is a continuous distribution with quantities from 0 to infinity. Because it is continuous, it is described by a probability density function (PDF) rather than a probability mass function (PMF). But when we use an exponential distribution to compute the likelihood of the data, we can treat densities as unnormalized probabilities.

- The Poisson and exponential distributions are parameterized by an event rate, denoted $\lambda$ or `lam`.

- For the prior distribution of $\lambda$, I (A.B.D.) used a gamma distribution, which is a continuous distribution with quantities from 0 to infinity, but I (A.B.D.) approximated it with a discrete, bounded PMF. The gamma distribution has one parameter, denoted $\alpha$ or alpha, which is also its mean.

## Exercises

### Exercise 1

> In the 2014 FIFA World Cup, Germany played Brazil in a semifinal match. Germany scored after 11 minutes and again at the 23 minute mark. At that point in the match, how many goals would you expect Germany to score after 90 minutes? What was the probability that they would score 5 more goals (as, in fact, they did)?

Here are the steps I (A.D.) recommend:

1. Starting with the same gamma prior we used in the previous problem, compute the likelihood of scoring a goal after 11 minutes for each possible value of lam. Don’t forget to convert all times into games rather than minutes.

2. Compute the posterior distribution of lam for Germany after the first goal.

3. Compute the likelihood of scoring another goal after 12 more minutes and do another update. Plot the prior, posterior after one goal, and posterior after two goals.

4. Compute the posterior predictive distribution of goals Germany might score during the remaining time in the game, 90-23 minutes. Note: You will have to think about how to generate predicted goals for a fraction of a game.

5. Compute the probability of scoring 5 or more goals during the remaining time.